In [1]:
# Install and load the necessary libraries
if (!requireNamespace("readr", quietly = TRUE)) {
  install.packages("readr")
}

# Replace 'path/to/census+income.zip' with the actual path to your local zip file
zip_file_path <- "census+income.zip"

# Replace 'path/to/extracted/files' with the desired directory to extract files
extracted_dir <- "data/"

# Unzip the file
unzip(zip_file_path, exdir = extracted_dir)

# Read the adult.data file into a dataframe
library(readr)
adult_data_path <- file.path(extracted_dir, "adult.data")
adult_df <- read.table(adult_data_path, header = FALSE, sep = ",", quote = "\"")

# Display the first few rows of the dataframe
head(adult_df)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
